<a href="https://colab.research.google.com/github/EmineKurucu/LLMProject/blob/main/LLMProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Kütüphaneler eklendi**

In [1]:
#!pip install requests
#!pip install PyPDF2
#!pip install transformers
#!pip install gradio
import os
import re
import time
import requests
from PyPDF2 import PdfReader
from transformers import AutoTokenizer
from tqdm import tqdm
import gradio as gr


In [2]:
# Hugging Face API ayalamalarını yapıyoruz
from google.colab import userdata
secretAPI = userdata.get('HF_API_TOKEN')
HF_API_TOKEN = secretAPI
API_URL = "https://api-inference.huggingface.co/models/meta-llama/Llama-3.1-8B-Instruct"
headers = {"Authorization" : f"Bearer {HF_API_TOKEN}"}

# Tokinizer çağırma
tokenizer = AutoTokinizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
max_token_limmit = 2048 # Model bağlamına göre ayarlandı.



SyntaxError: invalid syntax (<ipython-input-2-43ca3bd5d6ca>, line 2)

In [ ]:
# metni ayarlama
def extarct_text_from_pdf(pdf_file):
    # file'dan metni çıkarıyoruz
    try:
        reader = PdfReader(pdf_file)
        text = " "
        for page in reader.pages:
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Error extarcting text from the PDF: {str(e)}"

# text'te yer alan gereksiz boşlukları kaldırıyoruz
def clean_text(text):
    text = re.sub(r'\s+', ' ', text )
    text = re.sub('\b\d+\b\s*$', '', text, flags = re.MULTILINE )
    return text.strip()





In [ ]:
# text'i token'lara göre parçalara ayırıyoruz
def chunk_text(text, max_tokens = max_token_limit - 500):
  tokens = tokenzier.encode(text)
  chunks = []

  for i in range(0, len(tokens), max_tokens):
    chunk_tokens = tokens[i:i + max_tokens]
    chunk_text = tokenizer.decode(chunk_tokens)
    chunks.append(chunk_text)

  return chunks

In [ ]:
# eğitimin  'instructions' larını yapılandırıyoruz
def create_instruction_prompt(text_chunk, is_first_chunk = False, is_last_chunk = False ):
  system_prompt = """
    You are an expert educational content developer. Transform the following text, which is part of a casual transcript, into a formal, structured educational document suitable for a classroom lecture.

    Follow these guidelines:
    1. Convert casual language to formal, academic language
    2. Remove filler words, repetitions, and irrelevant tangents
    3. Organize content logically with clear sections and transitions
    4. Include relevant examples, case studies, or illustrations
    5. Maintain the core technical information and expertise
    6. Add appropriate educational elements (learning objectives, summaries, key points)

    The output should be detailed, coherent, and structured like a professional teaching transcript.
    """
    if is_first_chunk:
      chunk_context = "This is the beginning of the transkript. Start with an appropriate introduction."
    elif is_last_chunk :
      chunk_context = "This is the end of the transkript.Include a proper conclusion and summary."
    else:
      chunk_context = "This is a continuation of the transkript. Maintain consistency with previous context."

    prompt = f"<|system|>|n{system_prompt}\n\n{chunk_context}\n\n<|user|>\nTransform this transcript excerpt into educational content:\n\n{text_chunk}\n\n<|assistant|>"

    return prompt



In [ ]:
# alınan promptu Llama-3 modeline gönderme
# amaç LLaMA 3 dil modelini kullanmak ve API üzerinden cevap almak

def query_llama(prompt):
  max_retries = 5
  retry_delay = 10

  for attempt in range(max_retries):
    try:
      response = requests.post(
          API_URL,
          headers = headers,
          json = {"inputs": prompt, "parameters: {"max_new_tokens}: 2048"}

      )

      if response.status_code == 200:
        return response.json()[0]["generated_text"].split("<|assistant|>")[-1].strip()
      elif response.status_code == 503:
        print(f"Model is loading. Retrying in {retry_delay} seconds...")
        time.sleep(retry_delay)
      else:
        print(f"Error:" {response.status_code}, {response.text})
        return f"Error: response.status_code{})

    except Exception as e:
      print(f"Request error: {str(e)}")
      if attempt < max_retries -1:
        print(f"Retrying in {retry_delay} seconds...")
        time.sleep(retry_delay)
      else.
      return f"Failed after {max_retries} attempts: {str(e)}"
  return "Failed to get response from the model."





In [ ]:
# transkiptin akademik ders içeriğine dönüştürülmesi
def transform_transcript(pdf_file, lecture_length = "30min"):
  # metni ayrıştırma ve temizleme
  raw_text = extract_text_from_pdf(pdf_file)
  cleaned_text = clean_text(raw_text)

  # metni parçalara ayırma
  chunks = chunk_text(cleaned_text)

  results = []
  for i , chunk in enumarate(tqdm(chunks)):
    is_first = (i ==0)
    is_last = (i == len(chunks) -1 )

    prompt = create_educational_prompt(chunk, is_first, is_last)
    response = querly_llama(prompt)
    results.append(response)

    if i < len(chunks) -1 :
      time.sleep(2)

    # her chunk için alınan result'ları birleştirme.
    combined_result = "\n".join(results)

    if lecture_length == "30min":
      target_words = 3900
    else: # 60 min kısım için
      target_words = 9000

    refinement_prompts = f"""
    <|system|>
    You are an expert educational content developer. Review and refine the educational transcript below to create a cohesive, well-structured lecture.

    Your task is to:
    1. Ensure logical flow and consistent formatting
    2. Eliminate any redundancies or repetitions
    3. Add clear section headings and transitions
    4. Format the content appropriately for a {lecture_length} lecture (approximately {target_words} words)
    5. Add an introduction with learning objectives and a conclusion with key takeaways

    Create a professional, educational transcript that could be used directly by an instructor.

    <|user|>
    Please refine this draft educational transcript:

    {combined_result}

    <|assistant|>
    """

    # son halini alıyoruz
    final_result = query_llama(refinement_prompt)

    return final_result



**User Interface Kısmı**


In [ ]:
def process_pdf(pdf_file, lecture_length):
  if pdf_file is None:
    return "Please upload a PDF file."

  try:
    result = trnasform_transcript(pdf_file.name, lecture_length)
    return result

  except Exeption as e:
    return f"An error occured: {str(e)}"

# arayüzün tanımlanması
demo = gr.Inetrface(
    fn = process_pdf,
    inputs = [
        gr.File(label = "Upload PDF Transcript"),
        gr.Radio(label = "Lecture Length", choices = ["30min", "60min"], value = "30min")

    ],
    outputs = gr.Textbox(label = "Educational Transcript", lines = 20),
    title = "PDF Transcript to Educational Content Converter",
    description = "Upload a PDF transcript to transform it into structured educational content suitable for classroom teaching.",
    examples = [],
    cache_examples = False,
)



**Uygulamanın başlatılması**

In [ ]:
if __name__ == "__main__":
  demo.launch(share = True)